# EDA
En este notebook se hará un análisis exploratorio para poder tener una información mínima de los datos y se planteará como objetivo la reducción de la cantidad de datos debido a que se hara un deploy en la capa gratuita de Render y esta no cuenta con una gran catidad de recursos a nivel computacional.

## Análisis del archivo steam_games

In [1]:
import pandas as pd
import numpy as np
import gzip
import re
import ast

In [42]:
with gzip.open(r'E:\Data Science\PI1\Datasets\steam_games.json.gz','rt',encoding='utf-8') as f:
    df_games= pd.read_json(f, lines=True)

In [3]:
df_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


Se ve primero un vistazo general del dataframe y se evaluará que columnas son utiles para el MVP.

In [4]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


Como primer paso se eliminan los datos que no aportan nada de información.

In [43]:
df_games.drop(['publisher','url','specs','early_access','reviews_url'],axis=1,inplace=True)

Se ve como alrededor del 25% del total de entradas tienen información, por lo que se eliminan las filas vacias a continuación.

In [44]:
df_games=df_games.dropna(how='all')

Veremos ahora la cantidad de nulos

In [45]:
df_games.isnull().sum()

genres          3283
app_name           2
title           2050
release_date    2067
tags             163
price           1377
id                 2
developer       3299
dtype: int64

Primero se va a eliminar las filas donde los campos tengan pocos datos faltantes como *app_name*,*tags* e *id*.

In [46]:
df_games.dropna(subset=['app_name','tags','id'],inplace=True)

Por simplicidad y debido a que es un MVP se borrarán los nulos de las columnas *genres*, *release_date*, *price* y *developer*.  

In [47]:
df_games.dropna(subset=['genres','release_date','price','developer'],inplace=True)

Se verifica a continuación el estado del daframe para verificar si sigue habiendo datos nulos.

In [48]:
df_games.isnull().sum()

genres          0
app_name        0
title           0
release_date    0
tags            0
price           0
id              0
developer       0
dtype: int64

Ahora se comparan las columnas de *app_name* y *title* para ver su similitud y se tiene datos redundantes.

In [49]:
df_games[df_games['app_name']!=df_games['title']]

,genres,app_name,title,release_date,tags,price,id,developer
88390,"[Action, Adventure]",Sam & Max 101: Culture Shock,Sam &amp; Max 101: Culture Shock,2006-10-17,"[Point & Click, Comedy, Adventure, Detective, ...",19.99,8200.0,Telltale Games
88393,"[Action, Adventure]",Sam & Max 102: Situation: Comedy,Sam &amp; Max 102: Situation: Comedy,2006-12-20,"[Adventure, Action]",19.99,8210.0,Telltale Games
88419,[Strategy],Command & Conquer: Red Alert 3,Command &amp; Conquer: Red Alert 3,2008-10-28,"[Strategy, RTS, Base Building, Multiplayer, Co...",19.99,17480.0,EA Los Angeles
88492,[Strategy],Heroes of Might & Magic V: Hammers of Fate,Heroes of Might &amp; Magic V: Hammers of Fate,2006-11-14,"[Strategy, Turn-Based Strategy, Turn-Based, Fa...",9.99,15380.0,Nival
88494,[Strategy],Heroes of Might & Magic V: Tribes of the East,Heroes of Might &amp; Magic V: Tribes of the East,2007-10-16,"[Strategy, Turn-Based Strategy, Turn-Based, Fa...",9.99,15370.0,Nival
...,...,...,...,...,...,...,...,...
120181,"[Action, Adventure]",Sam & Max 105: Reality 2.0,Sam &amp; Max 105: Reality 2.0,2007-03-29,"[Adventure, Action]",19.99,8240.0,Telltale Games
120182,"[Action, Adventure]",Sam & Max 104: Abe Lincoln Must Die!,Sam &amp; Max 104: Abe Lincoln Must Die!,2007-02-22,"[Adventure, Action, Point & Click, Free to Pla...",19.99,8230.0,Telltale Games
120183,"[Action, Adventure]",Sam & Max 106: Bright Side of the Moon,Sam &amp; Max 106: Bright Side of the Moon,2007-04-26,"[Adventure, Action]",19.99,8250.0,Telltale Games
120208,"[Action, Strategy]",Making History: The Calm & the Storm,Making History: The Calm &amp; the Storm,2007-03-13,"[Strategy, Action, Turn-Based Strategy, Grand ...",4.99,6250.0,Muzzy Lane


Se va a eliminar la columna *title* debido a la cantidad de erroes basado solo con una simple inspección visual

In [50]:
df_games.drop('title',axis=1,inplace=True)

In [51]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27449 entries, 88310 to 120443
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        27449 non-null  object 
 1   app_name      27449 non-null  object 
 2   release_date  27449 non-null  object 
 3   tags          27449 non-null  object 
 4   price         27449 non-null  object 
 5   id            27449 non-null  float64
 6   developer     27449 non-null  object 
dtypes: float64(1), object(6)
memory usage: 1.7+ MB


Se verificara que no haya duplicados en las columnas

In [52]:
df_games['id'].duplicated().sum()

1

In [53]:
df_games.drop_duplicates(subset='id',inplace=True)

A la hora de elimnar duplicados en los nombres de las aplicaciones debemos tener cuidado que no pertenezcan a diferentes desarrolladores por lo que solo eliminaremos los duplicados que tambienpertenezcan a los mismos desarrolladores.

In [56]:
df_games[df_games[['app_name','developer']].duplicated(keep=False)]

,genres,app_name,release_date,tags,price,id,developer
92750,"[Adventure, Indie]",Invisible Apartment 2,2015-09-24,"[Adventure, Indie, Visual Novel, Anime, Hackin...",7.99,401200.0,"Milan Kazarka,Jeroen van Oosten"
96427,[Sports],Total Extreme Wrestling,2016-12-02,"[Sports, Wrestling, Management]",19.99,529700.0,Grey Dog Software
101465,"[Casual, Indie, Simulation]",MORE SWEATER? OK!,2017-12-19,"[Indie, Casual, Simulation]",0.99,773080.0,Triple-Star Studio
109299,"[Adventure, Free to Play, Indie]",Invisible Apartment 2,2016-10-17,"[Adventure, Indie, Free to Play, Visual Novel,...",7.99,543220.0,"Milan Kazarka,Jeroen van Oosten"
115399,"[Simulation, Sports]",Total Extreme Wrestling,2015-03-19,"[Wrestling, Simulation, Sports, Management]",19.99,344810.0,Grey Dog Software
120307,"[Action, Casual, Indie, RPG, Simulation]",MORE SWEATER? OK!,2017-12-29,"[Casual, Action, Indie, RPG, Simulation, Singl...",1.99,746050.0,Triple-Star Studio


In [57]:
df_games.drop_duplicates(subset=['app_name','developer'],inplace=True)

In [58]:
df_games[['app_name','developer']].duplicated().sum()

0

Se modificará el campo *release_date* ya que solo se necesita el año por lo que se procede a crear una función para modificar esta columna

In [59]:
#ESe eliminan los valores SOON, SOON™ y coming soon de la columna 'release_date' que son juegos que están por salir
# Se crea una lista con los valores a eliminar
valores = ['SOON', 'SOON™','coming soon']

# Eliminar las filas donde 'release_date' es 'SOON' o 'SOON™'
df_games = df_games[~df_games['release_date'].isin(valores)]

In [60]:
#Se crea un función para extraer los 4 primeros o ultimos caracteres de la columna 'release_date'
def extract_four_numbers(s):
    if re.match('^\\d{4}', s):
        return s[:4]  
    else:
        return s[-4:]  

# Aplica la función a la columna 'release_date' y crea una nueva columna 'release_year'
df_games['release_year'] = df_games['release_date'].apply(extract_four_numbers)

Se elimina la columna *release_date* ya que es innecesaria.

In [61]:
df_games.drop('release_date',axis=1,inplace=True)

Ahora se va a modificar la columna *price* para cambiar los strings de *free to play* a 0.

In [62]:
df_games['price'] = df_games['price'].apply(lambda x: 0 if isinstance(x, str) else x)

Se verifica que no haya mas strings.

In [63]:
df_games['price'].apply(type).value_counts()

price
<class 'float'>    27442
Name: count, dtype: int64

Se normanliza *app_name* y *developer* pcon la funcion title()

In [64]:
df_games['app_name']=df_games['app_name'].apply(lambda x: x.title())

In [65]:
df_games['developer']=df_games['developer'].apply(lambda x: x.title())

Se proceden a cambiar los tipos de datos para que esten acordes a los datos de las columnas.


In [66]:
#Primero se verifica el valor maximo de la columna id
df_games['id'].max()

2028850.0

In [67]:
df_games[['id', 'price']] = df_games[['id', 'price']].astype({'id': 'int32', 'price': 'float16'})

In [68]:
df_games['price']=df_games['price'].apply(lambda x: round(x,2))

Se guardan los datos limpios en formato parquet.

In [69]:
df_games.to_parquet(r'Datasets\steam_games_clean.parquet',index=False)

## Análisis del archivo users_items

In [6]:
data1 = []

with gzip.open(r'E:\Data Science\PI1\Datasets\users_items.json.gz', 'rt', encoding='utf-8') as file:
    for line in file:
        try:
            json_data = ast.literal_eval(line)
            data1.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

df_users = pd.DataFrame(data1)

In [7]:
df_users

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [8]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


En primer lugar se elimninan las columnas que no brindan información como *steam_id* y *user_url*.

In [9]:
df_users.drop(['steam_id','user_url'],axis=1,inplace=True)

Se desanidan el campo items

In [10]:
#Se expnade el json que esta en la columna 'items'
df_expandido=df_users.explode('items')

#Se normaliza ese dataframe
df_descompuesto=pd.json_normalize(df_expandido['items'])

#Se elimina las columnas que no se van a utilizar
df_descompuesto.drop(['item_name','playtime_2weeks'],axis=1,inplace=True)

#Se unen los dos dataframes anteriores
df_final=pd.concat([df_expandido.reset_index(drop=True), df_descompuesto.reset_index(drop=True)], axis=1)

#Se elimina la columna que se expandió
df_users=df_final.drop('items', axis=1)

Se ve el la gran cantidad de registros que posee el dataframe por lo que es indispensable reducirlo para utilizar solo lo que se necesita

In [11]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   item_id           object 
 3   playtime_forever  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 157.8+ MB


In [12]:
df_users.isnull().sum()

user_id                 0
items_count             0
item_id             16806
playtime_forever    16806
dtype: int64

Si no se tiene el id del juego va a ser imosible relacionarlo con otro dataset que tenga los id de los juegos por lo que los valores faltantes se van a tener que eliminar

In [13]:
df_users.dropna(subset='item_id',inplace=True)

Se analiza los datos de las horas de juegos para buscar valores atípicos

In [14]:
df_users['playtime_forever'].describe()

count    5.153209e+06
mean     9.914951e+02
std      5.418204e+03
min      0.000000e+00
25%      0.000000e+00
50%      3.400000e+01
75%      3.550000e+02
max      6.427730e+05
Name: playtime_forever, dtype: float64

Se eliminan el 25 porciento de los datos del 4º cuartil porque el estudio se va a centrar en los valores más usuales dentro de los gamers, es importante notar como la media está más a la derecha que la mediana

In [40]:
df_users=df_users[df_users['playtime_forever']<=355]

In [43]:
df_users=df_users.astype({'item_id':'int32','playtime_forever':'float16', 'items_count':'int16'})

Es clara la diferencia entre el dataframe al princio y el dataframe al final donde se redujo considerablemente el uso de la ram el cual va a ser un factor importante para el desempeño en Render

In [45]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3865427 entries, 0 to 5170013
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int16  
 2   item_id           int32  
 3   playtime_forever  float16
dtypes: float16(1), int16(1), int32(1), object(1)
memory usage: 88.5+ MB


In [46]:
df_users.to_parquet(r'Datasets\users_items_clean.parquet',index=False)

## Análisis del archivo user_revies

In [47]:
data2=[]

with gzip.open(r'E:\Data Science\PI1\Datasets\user_reviews.json.gz', 'rt', encoding='utf-8') as file:
    for line in file:
        try:
            json_data = ast.literal_eval(line)
            data2.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

df_reviews = pd.DataFrame(data2)

In [48]:
df_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


Se elimina la columna *user_url* y luego se desanida la columna review

In [49]:
df_reviews.drop('user_url',axis=1,inplace=True)

In [54]:
#Se expnade el json que esta en la columna 'reviews'
df_expandido=df_reviews.explode('reviews')

#Se normaliza ese dataframe
df_descompuesto=pd.json_normalize(df_expandido['reviews'])

#Se elimina las columnas que no se van a utilizar
df_descompuesto.drop(['funny','posted','last_edited','helpful'],axis=1,inplace=True)

#Se unen los dos dataframes anteriores
df_final=pd.concat([df_expandido.reset_index(drop=True), df_descompuesto.reset_index(drop=True)], axis=1)

#Se elimina la columna que se expandió
df_reviews=df_final.drop('reviews', axis=1)

In [56]:
df_reviews.isnull().sum()

user_id       0
item_id      28
recommend    28
review       28
dtype: int64

Se procede a eliminar los nulos en *items_id* ya que sin las claves de esos juegos no se puede hacer nada.

In [57]:
df_reviews.dropna(subset=['item_id'],inplace=True)

In [60]:
df_reviews.isnull().sum()

user_id      0
item_id      0
recommend    0
review       0
dtype: int64

In [62]:
df_reviews['item_id'].max()

'99910'

In [63]:
df_reviews['item_id']=df_reviews['item_id'].astype('int32')

In [64]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59305 entries, 0 to 59332
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    59305 non-null  object
 1   item_id    59305 non-null  int32 
 2   recommend  59305 non-null  object
 3   review     59305 non-null  object
dtypes: int32(1), object(3)
memory usage: 2.0+ MB


In [65]:
df_reviews.to_parquet(r'Datasets\user_reviews_clean.parquet',index=False)